# Scenario Integration Test

Лаботоратория по искусственному интеллекту, Сбербанк. 

О чем: вызов сценариев с разными моделями.
В качестве датасета используется датасет MovieLens100K. 

## Содержание

1. [Импорты, создание спарк-сессии](#intro)
2. [Загрузка данных](#data-loader)
3. [Сценарии с разными моделями](#scenario)
3.1 [Получение сценария через фабрику](#get-scenario)
3.2 [Обучение сценария](#fit-scenario)

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf

In [3]:
parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [19]:
from sponge_bob_magic.data_loader.datasets import download_movielens
from sponge_bob_magic.data_preparator.data_preparator import DataPreparator

from sponge_bob_magic.splitters import log_splitter
from sponge_bob_magic.splitters import user_log_splitter
from sponge_bob_magic.metrics import metrics

from sponge_bob_magic.models.popular_recomennder import PopularRecommender
from sponge_bob_magic.models.als_recommender import ALSRecommender
from sponge_bob_magic.models.knn_recommender import KNNRecommender
from sponge_bob_magic.models.lightfm_recommender import LightFMRecommender

from sponge_bob_magic.scenarios.main_scenario.main_factory import MainScenarioFactory

In [5]:
# отображение максимальной ширины колонок в pandas датафреймах
pd.options.display.max_colwidth = -1

In [6]:
spark_memory = "5g"
spark_cores = "*"
user_home = os.environ["HOME"]

spark = (
    SparkSession
    .builder
    .config("spark.driver.memory", spark_memory)
    .config("spark.local.dir", os.path.join(user_home, "tmp"))
    .master(f"local[{spark_cores}]")
    .enableHiveSupport()
    .getOrCreate()
)

spark

In [7]:
spark_logger = logging.getLogger("py4j")
spark_logger.setLevel(logging.WARN)

In [8]:
logger = logging.getLogger()
formatter = logging.Formatter("%(asctime)s, %(name)s, %(levelname)s: %(message)s",
                              datefmt="%d-%b-%y %H:%M:%S")
hdlr = logging.StreamHandler()
hdlr.setFormatter(formatter)
logger.addHandler(hdlr)
logger.setLevel(logging.DEBUG)

## Загрузка данных <a name="data-loader"></a>

In [9]:
path_data = os.path.join(os.environ["HOME"], "data")

if not os.path.exists(path_data):
    os.mkdir(path_data)
    
if not os.path.exists(os.path.join(path_data, "ml-100k")):
    download_movielens(path_data, "ml-100k")

## Подготовка данных <a name="data-preparator"></a>

In [10]:
path_log = os.path.join(path_data, "ml-100k", "u.data")

In [11]:
dp = DataPreparator(spark)

log = dp.transform_log(
    path_log,
    format_type="csv",
    columns_names={
        "user_id": "_c0", 
        "item_id": "_c1",
        "timestamp": "_c3"
    },
    sep="\t",
    date_format=None,
    header=False
)

In [12]:
log.show(3)

+-------+-------+-------------------+----------+---------+
|user_id|item_id|          timestamp|   context|relevance|
+-------+-------+-------------------+----------+---------+
|    196|    242|1997-12-04 18:55:49|no_context|      1.0|
|    186|    302|1998-04-04 23:22:22|no_context|      1.0|
|     22|    377|1997-11-07 10:18:36|no_context|      1.0|
+-------+-------+-------------------+----------+---------+
only showing top 3 rows


In [13]:
log.count()

100000

In [14]:
log.select([
    sf.count(sf.when(sf.col(c).isNull(), c)).alias(c) 
    for c in log.columns
]).show()

+-------+-------+---------+-------+---------+
|user_id|item_id|timestamp|context|relevance|
+-------+-------+---------+-------+---------+
|      0|      0|        0|      0|        0|
+-------+-------+---------+-------+---------+


In [15]:
log.agg(*(sf.countDistinct(sf.col(c)).alias(c) for c in log.columns)).show()

+-------+-------+---------+-------+---------+
|user_id|item_id|timestamp|context|relevance|
+-------+-------+---------+-------+---------+
|    943|   1682|    49282|      1|        1|
+-------+-------+---------+-------+---------+


In [16]:
log.agg(sf.min(sf.col("timestamp")), sf.max(sf.col("timestamp"))).show()

+-------------------+-------------------+
|     min(timestamp)|     max(timestamp)|
+-------------------+-------------------+
|1997-09-20 07:05:10|1998-04-23 03:10:38|
+-------------------+-------------------+


In [17]:
avg_num_users = (
    log
    .select("user_id", "item_id")
    .groupBy("item_id")
    .count()
    .select(sf.mean(sf.col("count")).alias("mean"))
    .collect()[0]["mean"]
)

avg_num_users

59.45303210463734

## Сценарии с разными моделями <a name="scenario"></a>

### Получение сценария через фабрику <a name="get-scenario"></a>

In [20]:
popular_recommender = PopularRecommender(spark)
als_recommender = ALSRecommender(spark)
knn_recommender = KNNRecommender(spark)
lightfm_recommender = LightFMRecommender(spark)

In [21]:
log_bydate_splitter = log_splitter.LogSplitByDateSplitter(
    spark, True, True,
    datetime(2007, 1, 1)
)

log_random_splitter = log_splitter.LogSplitRandomlySplitter(
    spark, True, True,
    test_size=0.3, seed=1234
)

log_cold_splitter = log_splitter.ColdUsersExtractingSplitter(
    spark, True, True,
    test_size=0.3
)

user_random_splitter = user_log_splitter.RandomUserLogSplitter(
    spark, True, True,
    item_test_size=0.3,
    seed=1234,
    user_test_size=500
)

user_bydate_splitter = user_log_splitter.ByTimeUserLogSplitter(
    spark, True, True,
    item_test_size=0.3,
    seed=1234,
    user_test_size=500
)

In [20]:
factory = MainScenarioFactory(spark)

### Обучение сценария <a name="fit-scenario"></a>

In [22]:
results = None

In [32]:
scenario = factory.get(
    splitter=user_random_splitter,
    recommender=knn_recommender,
    criterion=metrics.HitRateMetric(spark),
    metrics=[
        metrics.NDCGMetric(spark),
        metrics.PrecisionMetric(spark),
        metrics.MAPMetric(spark),
        metrics.RecallMetric(spark),
        metrics.Surprisal(spark, log),
    ],
    fallback_recommender=popular_recommender,
)

In [23]:
popular_grid = {
    "alpha": {"type": "int", "args": [0, 10]},
    "beta": {"type": "int", "args": [0, 10]}
}

als_grid = {
    "rank": {"type": "discrete_uniform", "args": [10, 100, 10]}
}

lightfm_grid = {
    "rank": {"type": "int", "args": [10, 100]}
}

knn_grid = {
    "shrink": {"type": "discrete_uniform", "args": [10, 50, 10]},
    "num_neighbours": {"type": "discrete_uniform", "args": [0, 10, 1]},
}

In [24]:
best_params = scenario.research(
    lightfm_grid,
    log,
    k=10,
    n_trials=2
)

21-Jan-20 10:13:26, root, DEBUG: Деление лога на обучающую и тестовую выборку
21-Jan-20 10:13:42, root, DEBUG: Длина трейна и теста: (84407, 15571)
21-Jan-20 10:13:44, root, DEBUG: Количество пользователей в трейне и тесте: 943, 494
21-Jan-20 10:13:45, root, DEBUG: Количество объектов в трейне и тесте: 1661, 1329
21-Jan-20 10:13:45, root, DEBUG: Обучение и предсказание дополнительной модели
21-Jan-20 10:13:45, root, DEBUG: Проверка датафреймов
21-Jan-20 10:13:46, root, DEBUG: Предварительная стадия обучения (pre-fit)
21-Jan-20 10:13:47, root, DEBUG: Среднее количество items у каждого user: 90
21-Jan-20 10:13:49, root, DEBUG: Основная стадия обучения (fit)
21-Jan-20 10:13:49, root, DEBUG: Проверка датафреймов
21-Jan-20 10:13:54, root, DEBUG: Количество items после фильтрации: 100
21-Jan-20 10:13:58, root, DEBUG: Пре-фит модели
21-Jan-20 10:14:11, root, DEBUG: -------------
21-Jan-20 10:14:11, root, DEBUG: Оптимизация параметров
21-Jan-20 10:14:11, root, DEBUG: Максимальное количество по

In [25]:
results = pd.concat([scenario.study.trials_dataframe(), results], axis=0)

results

number                state     value             datetime_start  \
                                                                     
0  0      TrialState.COMPLETE  0.846154 2020-01-21 10:14:12.050222   
1  1      TrialState.COMPLETE  0.945344 2020-01-21 10:14:53.935059   

           datetime_complete         params        user_attrs              \
                             num_neighbours shrink      MAP@k Precision@k   
0 2020-01-21 10:14:53.791877  0.0            50.0   0.047297   0.258502     
1 2020-01-21 10:15:40.872384  7.0            20.0   0.102325   0.406680     

                                  system_attrs  
   Recall@K Surprisal@K    nDCG@k      _number  
0  0.110151  1.081149    0.278023  0            
1  0.195762  1.634319    0.457021  1

### Получение рекомендаций <a name="predict-scenario"></a>

In [26]:
recs = scenario.production(
    best_params, 
    log,
    users=None, 
    items=None,
    k=10
)

21-Jan-20 10:15:40, root, DEBUG: Проверка датафреймов
21-Jan-20 10:15:41, root, DEBUG: Предварительная стадия обучения (pre-fit)
21-Jan-20 10:16:02, root, DEBUG: Основная стадия обучения (fit)
21-Jan-20 10:16:08, root, DEBUG: Проверка датафреймов
21-Jan-20 10:16:08, root, DEBUG: Выделение дефолтных юзеров
21-Jan-20 10:16:08, root, DEBUG: Выделение дефолтных айтемов


In [27]:
recs.show()

+-------+-------+------------------+----------+
|user_id|item_id|         relevance|   context|
+-------+-------+------------------+----------+
|    114|    174|1.6172478199005127|no_context|
|    114|    135|1.3242319822311401|no_context|
|    114|    197|1.2539494037628174|no_context|
|    114|    435|1.0871384143829346|no_context|
|    114|    483|0.9926429986953735|no_context|
|    114|    211|0.9692991375923157|no_context|
|    114|    514|0.9686022400856018|no_context|
|    114|    191|0.9193177223205566|no_context|
|    114|    511|0.7889890074729919|no_context|
|    114|    194|0.7673931121826172|no_context|
|    242|    286|1.4957059621810913|no_context|
|    242|    258| 1.039796233177185|no_context|
|    242|    300|1.0174559354782104|no_context|
|    242|    294|0.9061682820320129|no_context|
|    242|    237|0.8817647695541382|no_context|
|    242|    289|0.8736652731895447|no_context|
|    242|    100|0.8656629920005798|no_context|
|    242|    269|0.8325066566467285|no_c